<a href="https://colab.research.google.com/github/dyogaharshitha/VAE-for-USDA-SR-legacy-dataset/blob/main/trnsfrm_meal_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meal plan generator

Author: [D Yoga Harshitha](https://linkedin.com/in/dyogaharshitha)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras-nlp
!pip install positional-encodings[tensorflow]
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import keras_nlp
import matplotlib.pyplot as plt


Using TensorFlow backend


Keys USDA

In [ ]:
usda_food = pd.read_csv(r'/content/drive/MyDrive/DietApp/Fooddata/usda_sr_all_foods.csv')
print(usda_food["Food Group"].unique())
usda_food = usda_food.fillna(0)
usda_food["Cereals"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Baked Foods" else 1 if x=="Breakfast Cereals" else 1 if x=="Grains and Pasta" else 0.5 if x=="Baby Foods" else 0)
usda_food["Fruits"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Fruits" else 0 )
usda_food["Vegetables"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Vegetables" else 0 )
usda_food["nuts"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Nuts and Seeds" else 0.3 if x=="Baby Foods" else 0)
usda_food["pulses"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Beans and Lentils" else 0.3 if x=="Baby Foods" else 0 )
usda_food["dairy"] = usda_food["Food Group"].apply(lambda x: 0.7 if x=="Dairy and Egg Products" else 0 )
usda_food["non-veg"] = usda_food["Food Group"].apply(lambda x: 1 if x=="Meats" else 1 if x=="Fish" else 0 )
usda_food["processd"] = usda_food["Food Group"].apply(lambda x: 1 if x=="Beverages" else 1 if x=="Fast Foods" else 1 if x=="Soups and Sauces" else 0 )
usda_food.drop(columns=["Food Group","name","ID","200 Calorie Weight (g)","PRAL score"], inplace= True)
usda_cols = usda_food.columns.to_list()
usda_cols = ["Cereals","Vegetables","nuts","pulses","dairy","non-veg","processd"]+usda_cols[:-8]
print("cholestol"+str(usda_cols.index("Cholesterol (mg)"))+"  vita mcg"+str(usda_cols.index("Vitamin A, RAE (mcg)"))+"  vit c"+str(usda_cols.index("Vitamin C (mg)")))
usda_food = usda_food[usda_cols]
print(usda_food.head()); print(usda_food.shape);

['Baked Foods' 'Snacks' 'Sweets' 'Vegetables' 'American Indian'
 'Restaurant Foods' 'Beverages' 'Fats and Oils' 'Meats'
 'Dairy and Egg Products' 'Baby Foods' 'Breakfast Cereals'
 'Soups and Sauces' 'Beans and Lentils' 'Fish' 'Fruits' nan
 'Grains and Pasta' 'Nuts and Seeds' 'Prepared Meals' 'Fast Foods'
 'Spices and Herbs' 'Dairy and Egg Products ']
cholestol13  vita mcg20  vit c21
   Cereals  Vegetables  nuts  pulses  dairy  non-veg  processd  Calories  \
0      0.7         0.0   0.0     0.0    0.0        0         0     307.0   
1      0.7         0.0   0.0     0.0    0.0        0         0     330.0   
2      0.7         0.0   0.0     0.0    0.0        0         0     377.0   
3      0.7         0.0   0.0     0.0    0.0        0         0     232.0   
4      0.7         0.0   0.0     0.0    0.0        0         0     273.0   

   Fat (g)  Protein (g)  ...  Histidine (mg)  Alanine (mg)  \
0    13.24         5.88  ...             0.0           0.0   
1    11.27         4.34  ...     

cluster and select

In [ ]:
from sklearn.cluster import KMeans

clstr = KMeans(500)
clstr.fit(usda_food)
usda_clstr = clstr.transform(usda_food)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
usda_clstr = usda_food.copy(deep=True)
usda_clstr['clustr']=clstr.labels_
usda_clstr = usda_clstr.groupby('clustr',as_index=False).nth(0)
usda_clstr.drop('clustr',axis=1,inplace=True)
usda_clstr

,Cereals,Vegetables,nuts,pulses,dairy,non-veg,processd,Calories,Fat (g),Protein (g),...,Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg)
0,0.7,0.0,0.0,0.0,0.0,0,0,307.0,13.24,5.88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.7,0.0,0.0,0.0,0.0,0,0,330.0,11.27,4.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.7,0.0,0.0,0.0,0.0,0,0,232.0,1.80,8.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.7,0.0,0.0,0.0,0.0,0,0,273.0,9.22,6.58,...,143.0,249.0,406.0,1614.0,214.0,559.0,347.0,0.0,0.0,0.0
6,0.7,0.0,0.0,0.0,0.0,0,0,289.0,9.40,6.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12695,0.0,0.0,0.0,0.0,0.0,0,0,529.0,32.19,2.92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12696,0.0,0.7,0.0,0.0,0.0,0,0,193.0,13.13,1.74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13804,0.0,0.0,0.0,0.0,0.0,0,0,764.0,86.30,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13813,0.0,0.0,0.0,0.0,0.0,0,0,594.0,64.20,0.90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Prepare queries :  the input spec for meal - age, diet type, cal req, meal type


In [ ]:
inp_spec = pd.read_csv("/content/drive/MyDrive/DietApp/Fooddata/nutri_req_cntrls.csv")
inp_spec = inp_spec.fillna(0)
inp_spec['Cereal'] = inp_spec['Cereal']/100
inp_spec['Vegetables'] = inp_spec['Vegetables']/100 ; inp_spec['nuts'] = inp_spec['nuts']/100 ; inp_spec['pulses'] = inp_spec['pulses']/100
inp_spec['dairy'] = inp_spec['dairy']/100 ; inp_spec['non-veg'] = inp_spec['non-veg']/100 ; inp_spec['processd'] = inp_spec['processd']/100
inp_qry = inp_spec.drop(['Unnamed: 0','total_qty_max'], axis=1)
inp_spec

,Unnamed: 0,total_qty_max,Cereal,Vegetables,nuts,pulses,dairy,non-veg,processd,Calories,...,Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg)
0,inf_0_6m,200.0,0.200,0.200,0.100,0.200,0.100,0.000,0.000,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,inf_6_12m,250.0,0.400,0.400,0.200,0.400,0.200,0.000,0.000,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ag1-3,500.0,0.400,0.400,0.100,0.400,0.400,0.000,0.100,1000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ag4-6,700.0,0.400,0.400,0.100,0.400,0.400,0.300,0.200,1200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ag7-9,800.0,0.400,0.400,0.100,0.400,0.400,0.300,0.300,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ag10-12,1000.0,0.500,0.500,0.100,0.500,0.500,0.300,0.300,1900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ag13-15,1200.0,0.500,0.500,0.100,0.500,0.500,0.300,0.300,2300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,ag16-17,1200.0,0.500,0.500,0.100,0.500,0.500,0.300,0.300,2500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,adlt_lowact,1500.0,2.500,2.000,0.100,0.500,1.000,1.000,0.100,1300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,adlt_medact,1500.0,0.050,2.000,0.400,0.900,1.000,2.000,0.100,1700.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Query to emb,target and mask mapping

0. age13-15 wholeday norm_cntrl
1. age16-17 wholeday norm_cntrl
2. adlt_lowact wholeday norm_cntrl
3. adlt_hghact wholeday norm_cntrl
4. age13-15 lunch norm_cntrl
5. age16-17 lunch norm_cntrl
6. adlt_lowact lunch norm_cntrl
7. adlt_hghact lunch norm_cntrl
8. age13-15 wholeday free_cntrl
9. age16-17 wholeday free_cntrl
10. adlt_lowact wholeday free_cntrl
11. adlt_hghact wholeday free_cntrl
12. age13-15 lunch free_cntrl
13. age16-17 lunch free_cntrl
14. adlt_lowact lunch free_cntrl
15. adlt_hghact lunch free_cntrl


In [ ]:
inp=3;
if inp < 8 :
  out_mask = np.array(inp_qry.iloc[24].values)
else:
  out_mask = np.array(inp_qry.iloc[25].values)
if inp % 4 == 0 :
  age = np.array(inp_qry.iloc[6].values)
elif inp % 4 == 1 :
  age = np.array(inp_qry.iloc[7].values)
elif inp%4 ==2:
  age = np.array(inp_qry.iloc[8].values)
else :
  age = np.array(inp_qry.iloc[10].values)
if inp % 7 < 4:
  lnch = np.array(inp_qry.iloc[15].values); print(age)
  tar = age * lnch * out_mask
else:
  tar = age * out_mask

def map_tarmsk(inp):
  if inp < 8 :
    out_mask = np.array(inp_qry.iloc[24].values)
  else:
    out_mask = np.array(inp_qry.iloc[25].values)
  if inp % 4 == 0 :
    age = np.array(inp_qry.iloc[6].values)
  elif inp % 4 == 1 :
    age = np.array(inp_qry.iloc[7].values)
  elif inp%4 ==2:
    age = np.array(inp_qry.iloc[8].values)
  else :
    age = np.array(inp_qry.iloc[10].values)
  if inp % 7 < 4:
    lnch = np.array(inp_qry.iloc[15].values)
    tar = age * lnch * out_mask
  else:
    tar = age * out_mask
  tar = np.reshape(tar,(1,100))
  out_mask = np.reshape(out_mask,(1,100))
  return tar, out_mask

def pos_enc(pos, dim, n=10000):
  p = np.zeros((1,dim))
  for i in np.arange(int(dim/2)):
    den = np.power(n,2*i/dim)
    p[0,2*i] = np.sin(pos/den)
    p[0,2*i+1] = np.cos(pos/den)
    return p




[3.25e+00 2.00e+00 4.00e-01 9.00e-01 1.00e+00 2.00e+00 1.00e-01 2.02e+03
 3.00e+01 7.50e+01 3.35e+02 0.00e+00 0.00e+00 2.00e+00 0.00e+00 5.00e+02
 3.00e+01 0.00e+00 0.00e+00 0.00e+00 7.70e+02 1.20e+02 5.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 1.00e+01 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00]


# Dataset

train data set preparation

input spec -- encoding

food data -- norm + pos encoding

target spec

loss function

In [ ]:
"""
pos_enc_df_3000 = pd.DataFrame()
dim=100
for i in range(3000):
  pos_enc_df_3000.iloc[i] = pos_enc(i, dim)
pos_enc.to_csv('/content/drive/MyDrive/DietApp/Fooddata/posencoding/pos_snc_df_3000_dim_100.csv')
"""

In [ ]:
btch = 320 ;  dim=100; key_len=500;  out_len=4

inp_seq = np.random.randint(0,high=16,size=(btch))
inp_seq_emb = tf.keras.layers.Embedding(input_dim=16,output_dim=dim)(inp_seq)
inp_seq_emb = tf.expand_dims(inp_seq_emb,axis=1)
inp_pos_btch = np.random.randint(0,high=4,size=(btch))
inp_pos_emb = tf.keras.layers.Embedding(input_dim=5,output_dim=out_len)(inp_pos_btch)
#inp_pos_emb = keras_nlp.layers.SinePositionEncoding()(np.zeros((btch,out_len)))
inp_pos_emb = np.random.rand(btch,out_len) * np.random.randint(0,10,size=(btch,out_len)) ; inp_pos_emb = inp_pos_emb.astype(np.float32)
inp_pos_emb = tf.expand_dims(inp_pos_emb,axis=-1)
pos_enc = tf.matmul(inp_pos_emb,inp_seq_emb)
inp_x = tf.math.add(inp_seq_emb,pos_enc)  # shape(btch,out_len,dim)
print(inp_x.shape)

key_pos_emb = keras_nlp.layers.SinePositionEncoding()(np.zeros((btch,key_len)))
key_pos_emb = tf.expand_dims(key_pos_emb, axis=-1)
key_pos = key_pos_emb * np.ones((btch,key_len,dim)) ; key_pos = tf.cast(key_pos,dtype=tf.float32)

kys = tf.expand_dims(usda_clstr,axis=0 ) ; kys = kys+np.zeros((btch,key_len,dim)); print(kys.shape)
kys = tf.cast(kys, tf.float32)
#kys = tf.math.add(kys , 0.5*key_pos)  ;
kys_norm = tf.keras.layers.BatchNormalization()(kys)

targt_y = np.zeros((btch,1,dim))
out_msk = np.zeros((btch,1,dim))
for i in range(btch):
  targt_y[i,0,:], out_msk[i,0,:] = map_tarmsk(i)


(320, 4, 100)
(320, 500, 100)


In [ ]:
 print(tf.keras.layers.Embedding(input_dim=15,output_dim=4)(3))
 keras_nlp.layers.SinePositionEncoding()(np.zeros((3,4)))

tf.Tensor([-0.00236504  0.01907114 -0.00523492  0.04349214], shape=(4,), dtype=float32)


<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.84147096,  0.5403023 ,  0.00999983,  0.99995   ],
       [ 0.9092974 , -0.41614684,  0.01999867,  0.9998    ]],
      dtype=float32)>

In [ ]:
btch = 320 ;  dim=100; key_len=500;  out_len=4

inp_seq1 = np.random.randint(0,high=16,size=(btch))
inp_seq_emb1 = tf.keras.layers.Embedding(input_dim=16,output_dim=dim)(inp_seq1)
inp_seq_emb1 = tf.expand_dims(inp_seq_emb1,axis=1)
inp_pos_btch1 = np.random.randint(0,high=5,size=(btch))
inp_pos_emb1 = tf.keras.layers.Embedding(input_dim=5,output_dim=out_len)(inp_pos_btch1)
inp_pos_emb1 = tf.expand_dims(inp_pos_emb1,axis=-1)
pos_enc1 = tf.matmul(inp_pos_emb1,inp_seq_emb1)
inp_x1 = tf.math.add(inp_seq_emb1,pos_enc1)  # shape(btch,out_len,dim)
print(inp_x1.shape)

targt_y1 = np.zeros((btch,1,dim))
out_msk1 = np.zeros((btch,1,dim))
for i in range(btch):
  targt_y1[i,0,:], out_msk1[i,0,:] = map_tarmsk(i)

inp_seq2 = np.random.randint(0,high=16,size=(btch))
inp_seq_emb2 = tf.keras.layers.Embedding(input_dim=16,output_dim=dim)(inp_seq2)
inp_seq_emb2 = tf.expand_dims(inp_seq_emb2,axis=1)
inp_pos_btch2 = np.random.randint(0,high=5,size=(btch))
inp_pos_emb2 = tf.keras.layers.Embedding(input_dim=5,output_dim=out_len)(inp_pos_btch2)
inp_pos_emb2 = tf.expand_dims(inp_pos_emb2,axis=-1)
pos_enc2 = tf.matmul(inp_pos_emb2,inp_seq_emb1)
inp_x2 = tf.math.add(inp_seq_emb2,pos_enc2)  # shape(btch,out_len,dim)
print(inp_x2.shape)

targt_y2 = np.zeros((btch,1,dim))
out_msk2 = np.zeros((btch,1,dim))
for i in range(btch):
  targt_y2[i,0,:], out_msk2[i,0,:] = map_tarmsk(i)


(320, 4, 100)
(320, 4, 100)


In [ ]:
from sklearn.preprocessing import Normalizer

kys = tf.cast(kys,tf.float32)
targt_y = tf.cast(targt_y, tf.float32)
targt_y1 = tf.cast(targt_y1, tf.float32)
targt_y2 = tf.cast(targt_y2, tf.float32)
targt_y

"""
kysnrm= Normalizer()
kys_norm = tf.keras.layers.BatchNormalization()(kys)
tarnrm = Normalizer()
targt_y = tf.keras.layers.BatchNormalization()(targt_y)
targt_y1 = tf.keras.layers.BatchNormalization()(targt_y1)
targt_y2 = tf.keras.layers.BatchNormalization()(targt_y2)
"""

'\nkysnrm= Normalizer()\nkys_norm = tf.keras.layers.BatchNormalization()(kys)\ntarnrm = Normalizer()\ntargt_y = tf.keras.layers.BatchNormalization()(targt_y)\ntargt_y1 = tf.keras.layers.BatchNormalization()(targt_y1)\ntargt_y2 = tf.keras.layers.BatchNormalization()(targt_y2)\n'

loss function

In [ ]:
out_seq = np.ones((btch,out_len,dim))

def get_loss(out_seq,msk,targt):
  out = tf.math.reduce_sum(out_seq,axis=1,keep_dim=True)
  out = out * msk
  loss_norm = out - targt / 25
  return loss_norm

# Transformer

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    # Shape `(batch_size, seq_len, d_model)
    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x, self.last_attn_scores



In [ ]:
arry = tf.constant([[[1.0,2.0],[2.0,3],[3,1]]])
ansre = tf.keras.activations.softmax(arry,axis=-2)
print(arry)
print(ansre)

tf.Tensor(
[[[1. 2.]
  [2. 3.]
  [3. 1.]]], shape=(1, 3, 2), dtype=float32)
tf.Tensor(
[[[0.09003057 0.24472848]
  [0.24472848 0.6652409 ]
  [0.6652409  0.09003057]]], shape=(1, 3, 2), dtype=float32)


In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    self.sing_head = DecoderLayer(d_model=d_model, num_heads=1,dff=dff, dropout_rate=dropout_rate)
    self.dens =  tf.keras.layers.Dense(200, activation='relu')
    self.dens_fnl =  tf.keras.layers.Dense(1,activation='tanh')
    self.dpt_conv = tf.keras.layers.Conv1D(4,50,strides=1,activation='tanh')
    self.btch_norm = tf.keras.layers.BatchNormalization(); self.btch_norm1 = tf.keras.layers.BatchNormalization(); self.btch_norm2 = tf.keras.layers.BatchNormalization()
    self.cal_msk = np.zeros((1,100)) ; self.cal_msk[0,7:15] = 1; self.cal_msk = tf.convert_to_tensor(self.cal_msk,dtype=tf.float32)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x1  = inputs


    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x, attn_scrs = self.decoder(x1, context)  # (batch_size, target_len, d_model)

    """add another decoder one one head attention"""
    x = self.sing_head(x,context)
    attn_scrs = self.sing_head.last_attn_scores

    # Final linear layer output.
    # logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    #for 4 dsh wgts
    logits = tf.transpose(tf.squeeze(attn_scrs,axis=1),perm=[0,2,1])

    logits = tf.keras.activations.softmax(logits,axis=1)
    logits = tf.matmul(logits,x1)
    logits = tf.math.multiply(logits, self.cal_msk)
    logits = tf.keras.layers.Concatenate()([logits,context])
    logits = self.btch_norm(logits)
    logits = self.dpt_conv(tf.transpose(logits,perm=[0,2,1]))
    logits = self.btch_norm1(logits)
    logits = self.dens(tf.transpose(logits,perm=[0,2,1]))
    logits = self.btch_norm2(logits)
    logits = self.dens_fnl(logits)
    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits, x, attn_scrs



Initialize the model

In [ ]:
qry_len = 4; key_len =500 ; dim=100;
num_heads= 10   ; num_layers=1 ;

mdl = Transformer( num_layers=3, d_model=dim, num_heads=10, dff=100,
               input_vocab_size=100, target_vocab_size=1, dropout_rate=0.1)

In [ ]:

inp = tf.expand_dims(inp_x[:32,:,:],axis=1) ;  ky = tf.expand_dims(kys[:32,:,:],axis=1)
out,x ,att = mdl([inp_x[:32,:,:],kys_norm[:32,:,:]])
print(out.shape,x.shape,att.shape)

(32, 4, 1) (32, 500, 100) (32, 1, 500, 4)


# Training

In [ ]:
def get_norm_loss(out_seq,msk,targt):
  out = tf.math.reduce_sum(out_seq,axis=1,keepdims=True)
  out = tf.math.multiply(out , msk)
  targt = tf.math.multiply(targt,msk)
  loss_norm = (out - targt) / 25 ; loss_norm = tf.math.abs(loss_norm)

  cal_msk = np.zeros((1,100)); cal_msk[0,7]=1
  cal_loss = tf.multiply(loss_norm,cal_msk)
  loss_sig = 1 - tf.keras.activations.sigmoid(out-targt)
  sig_mask = np.ones((1,100)) ; sig_mask[0,0:7] = 0.5 ; # sig_mask[0,10:16] = 0.5 ;
  sig_loss = tf.multiply(loss_sig,sig_mask)
  loss_tot = cal_loss + loss_sig
  return loss_tot

class tfrmr(tf.keras.Model):
    def __init__(self,mdl,qry,kys_norm,out_msk,targt,los,opt,tbl):
        super().__init__()
        self.mdl = mdl;
        self.qry = qry
        self.kys = kys_norm
        self.out_msk = out_msk
        self.targt = targt
        self.los=los;
        self.opt= opt;
        self.tbl= tbl
        #print(self.qry.shape, self.kys.shape, self.out_msk.shape, self.targt.shape, self.tbl.shape)

    def compile(self,loss,opt,metrics):
        super().compile()
        self.mdl.compile(loss=self.los,optimizer=self.opt,metrics=["Accuracy"])
    def train_step(self,inp):

        batch=32 ; loss=0 ; num_btch=1
        #qryitr = iter(self.qry) ; taritr = iter(self.targt)
        for i in range(num_btch):
            kys = self.kys[0:batch, :,:] ; out_msk = self.out_msk ; tbl = self.tbl[0:batch,:,:] ;
            qry = self.qry[i*batch: i*batch+batch,: ,:] ; targt = self.targt[i*batch:i*batch+batch,: ,:]
            with tf.GradientTape() as grdtp:
                grdtp.watch(self.mdl.trainable_variables)
                out_wgt, x, attn_scr = self.mdl([qry, kys])
                out = tf.math.argmax(attn_scr, axis=-2)
                out = tf.one_hot(out,500,axis=-1)
                out = tf.squeeze(out)
                #k = tf.transpose(self.kys,[0,2,1])
                out_seq = tf.matmul(out,tbl)
                out_seq = out_seq * out_wgt *2.5
                lss = self.los(out_seq,out_msk,targt)
                loss = loss+lss
                #cos_los = tf.keras.losses.cosine_similarity(self.rel,fke)
                #lss_tot = 10*tf.keras.losses.mean_absolute_error(self.rel,fke) +1.5*(1+cos_los)**2
                if i < num_btch-1 :
                  continue
                loss = loss/num_btch
                grd = grdtp.gradient(loss,self.mdl.trainable_variables)
            self.opt.apply_gradients(zip(grd,self.mdl.trainable_variables))
            """
            with tf.GradientTape() as grdtp:
                grdtp.watch(self.mdl.trainable_variables)
                out_wgt, x, attn_scr = self.mdl([inp_x1,self.kys ])
                out = tf.math.argmax(attn_scr, axis=-2)
                out = tf.one_hot(out,500,axis=-1)
                out = tf.squeeze(out)
                #k = tf.transpose(self.kys,[0,2,1])
                out_seq = tf.matmul(out,self.tbl)
                out_seq = out_seq * out_wgt *2
                lss1 = self.los(out_seq,out_msk1,targt_y1)
                #cos_los = tf.keras.losses.cosine_similarity(self.rel,fke)
                #lss_tot = 10*tf.keras.losses.mean_absolute_error(self.rel,fke) +1.5*(1+cos_los)**2

                grd = grdtp.gradient(lss1,self.mdl.trainable_variables)

            self.opt.apply_gradients(zip(grd,self.mdl.trainable_variables))

            with tf.GradientTape() as grdtp:
                grdtp.watch(self.mdl.trainable_variables)
                out_wgt, x, attn_scr = self.mdl([inp_x2, self.kys ])
                out = tf.math.argmax(attn_scr, axis=-2)
                out = tf.one_hot(out,500,axis=-1)
                out = tf.squeeze(out)
                #k = tf.transpose(self.kys,[0,2,1])
                out_seq = tf.matmul(out,self.tbl)
                out_seq = out_seq * out_wgt *2
                lss1 = self.los(out_seq,out_msk2,targt_y2)
                #cos_los = tf.keras.losses.cosine_similarity(self.rel,fke)
                #lss_tot = 10*tf.keras.losses.mean_absolute_error(self.rel,fke) +1.5*(1+cos_los)**2

                grd = grdtp.gradient(lss1,self.mdl.trainable_variables)

            self.opt.apply_gradients(zip(grd,self.mdl.trainable_variables)) """

        return {"loss":loss }






In [ ]:
opt = tf.keras.optimizers.Adam(0.00001)
#targt_y = tf.data.Dataset.from_tensors(targt_y) ; inp_x = tf.data.Dataset.from_tensors(inp_x)
dta = tf.data.Dataset.from_tensor_slices((inp_x,targt_y)) ;
trng = tfrmr(mdl,inp_x,kys_norm,out_mask,targt_y,get_norm_loss,opt,kys)
trng.compile(get_norm_loss,opt,['Accuracy'])
hist = trng.fit(dta, epochs=3, batch_size=32)

Epoch 1/3


320/320 [==============================] - 3999s 12s/step - loss: 0.6761
Epoch 2/3
320/320 [==============================] - 3977s 12s/step - loss: 0.6149
Epoch 3/3
 78/320 [======>.......................] - ETA: 50:08 - loss: 0.5891

In [ ]:
hist = trng.fit(inp_x, epochs=2)

Epoch 1/2
10/10 [==============================] - 11s 1s/step - loss: 0.7811
Epoch 2/2
10/10 [==============================] - 11s 1s/step - loss: 0.7824


In [ ]:
opt = tf.keras.optimizers.Adam(0.000001)
trng = tfrmr(mdl,inp_x,kys_norm,out_mask,targt_y,get_norm_loss,opt,kys)
trng.compile(get_norm_loss,opt,['Accuracy'])
hist = trng.fit(inp_x, epochs=5)

Epoch 1/5


10/10 [==============================] - 48s 1s/step - loss: 0.7833
Epoch 2/5
10/10 [==============================] - 11s 1s/step - loss: 0.7843
Epoch 3/5
10/10 [==============================] - 11s 1s/step - loss: 0.7846
Epoch 4/5
10/10 [==============================] - 11s 1s/step - loss: 0.7862
Epoch 5/5
10/10 [==============================] - 11s 1s/step - loss: 0.7845


In [ ]:
ky= kys[5:10,:,:] ; inp= inp_x[10:15,:,:];
out_wgt, x, attn_scr = mdl([inp, ky])
out1 = tf.math.argmax(attn_scr, axis=-2)
out = tf.one_hot(out1,500,axis=-1)
out = tf.squeeze(out)

out_seq = tf.matmul(out,ky)
out_seq = out_seq * out_wgt
print(out1); print(out_wgt)
print(tf.reduce_sum(out_seq,axis=-2))

tf.Tensor(
[[[174 443 443 174]]

 [[436 439 201 353]]

 [[455 496 455 496]]

 [[ 44 174 443 429]]

 [[144 201 144 144]]], shape=(5, 1, 4), dtype=int64)
tf.Tensor(
[[[0.49769044]
  [0.33997673]
  [0.39708725]
  [0.08886725]]

 [[0.23031364]
  [0.6138173 ]
  [0.07017171]
  [0.7975638 ]]

 [[0.6935474 ]
  [0.8106002 ]
  [0.73747325]
  [0.68074363]]

 [[0.53252727]
  [0.43229437]
  [0.18844831]
  [0.00192475]]

 [[0.6052524 ]
  [0.05927118]
  [0.6250216 ]
  [0.5523612 ]]], shape=(5, 4, 1), dtype=float32)
tf.Tensor(
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 5.86557686e-01 0.00000000e+00 7.39324524e+02
  7.91085968e+01 6.02981281e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.70762061e+03 1.78829956e+01 5.86557674e+00
  9.38492298e-01 1.51331879e+02 8.21180725e+00 7.37063984e+04
  2.21119180e+04 7.91852903e+00 1.28456140e+00 1.84265991e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 4.59626617e+01
  1.49396885e+04 7.41945007e+02 0.00000000e+00 0.

In [ ]:
usda_lbl = pd.read_csv(r'/content/drive/MyDrive/DietApp/Fooddata/usda_sr_all_foods.csv')
ky= kys[10:15,:,:] ; inp= inp_x[10:15,:,:]
out_wgt, x, attn_scr = mdl([inp, ky])
out1 = tf.math.argmax(attn_scr, axis=-2)
#out = tf.one_hot(out1,500,axis=-1)
out = tf.squeeze(out1)
for i in range(5):
  print("input spec: ",inp_seq[i])
  dsh = tf.get_static_value(out[i,0])
  print("meal plan: ", usda_lbl.iloc[dsh,1], out_wgt[i,0,0]," in 100 g")
  print( usda_lbl.iloc[tf.get_static_value(out[i,1]),1], out_wgt[i,1,0]," in 100 g")
  print( usda_lbl.iloc[tf.get_static_value(out[i,2]),1], out_wgt[i,2,0]," in 100 g")


input spec:  7
meal plan:  Milk Dessert Frozen Milk-Fat Free Chocolate tf.Tensor(0.49769044, shape=(), dtype=float32)  in 100 g
Snacks Granola Bars Soft Uncoated Peanut Butter tf.Tensor(0.33997673, shape=(), dtype=float32)  in 100 g
Snacks Granola Bars Soft Uncoated Peanut Butter tf.Tensor(0.39708725, shape=(), dtype=float32)  in 100 g
input spec:  4
meal plan:  Snacks Corn-Based Extruded Onion-Flavor tf.Tensor(0.23031364, shape=(), dtype=float32)  in 100 g
Snacks Crisped Rice Bar Chocolate Chip tf.Tensor(0.6138173, shape=(), dtype=float32)  in 100 g
Turkey Light Or Dark Meat Smoked Cooked Skin And Bone Removed tf.Tensor(0.070171714, shape=(), dtype=float32)  in 100 g
input spec:  12
meal plan:  Snacks Rice Cakes Brown Rice Buckwheat tf.Tensor(0.6935474, shape=(), dtype=float32)  in 100 g
Schar Gluten-Free Classic White Rolls tf.Tensor(0.8106002, shape=(), dtype=float32)  in 100 g
Snacks Rice Cakes Brown Rice Buckwheat tf.Tensor(0.73747325, shape=(), dtype=float32)  in 100 g
input spec

In [ ]:
arr = tf.constant([[[1,2,3],[2,5,1]],[[1,2,3],[4,1,2]]])
print(arr.shape)
arm = tf.math.argmax(arr,axis=-1)
print(arm)
arm = tf.one_hot(arm,3,axis=-1)
print(arm.shape)
arr = np.ones((2,2,3))  # ((2,3,4))
arm = tf.transpose(arm,[0,2,1])
print(arm.shape)
tf.matmul(arm, arr)




In [ ]:
x = np.array([[1,2,3],[4,5,6]])
x = np.reshape(x,(2,3,1))
y = np.ones((2,3,5))
print(x*y)

In [ ]:
x = np.ones((2,1,5))
y = np.ones((2,3,5))
tf.math.add(x,y)

In [ ]:
from positional_encodings.tf_encodings import TFPositionalEncoding2D

inp_seq = np.array([[[[1,2,3],[1,1,1]],[[1,1,1],[1,1,1]]]])
enc = TFPositionalEncoding2D(3)
inp_x = enc(inp_seq)
print(inp_x)
enc = tf.keras.layers.Embedding(input_dim=4,output_dim=4)(inp_seq)
pos = keras_nlp.layers.SinePositionEncoding()([[1,2,3]])
pos = tf.keras.layers.Embedding(input_dim=50,output_dim=3)(np.array([1,2,3]))
print(pos)